In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
html=requests.get(url)
soup=BeautifulSoup(html.text, 'html.parser')
table=soup.find('table',{'class':'wikitable sortable'})

In [4]:
columns = [th.text.replace('\n', '') for th in table.find('tr').find_all('th')]
trs = table.find_all('tr')[1:]
rows = list()
for tr in trs:
    rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])
df=pd.DataFrame(data=rows,columns=columns)

In [5]:
df.drop(df[df.Borough=='Not assigned'].index,inplace=True)
df['Neighbourhood'].replace('Not assigned',df['Borough'],inplace=True)

In [6]:
duf=df['Postcode'].duplicated(keep='first')
duf=list(duf[duf==True].index)
dul=df['Postcode'].duplicated(keep='last')
dul=list(dul[dul==True].index)
for i,j in zip(dul[::-1],duf[::-1]):
    df.loc[i,'Neighbourhood']=str(df.loc[j,'Neighbourhood'])+','+str(df.loc[i,'Neighbourhood'])
df.drop(index=duf,inplace=True)
df.sort_values('Postcode',inplace=True)
df.reset_index(drop=True,inplace=True)

In [7]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"West Hill,Morningside,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df.shape

(103, 3)

# Get Geo Data

In [9]:
df_geo=pd.read_csv('https://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df_new=df.merge(df_geo,left_on='Postcode',right_on='Postal Code',how='left')
df_new.drop('Postal Code',axis=1,inplace=True)
df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Morningside,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Clustering

In [11]:
# @hidden_cell
CLIENT_ID = 'TFBQ3TCNGGG12CQDSF0LJSXUVGADSDLUEVEDRXEUW0P21W0F' # your Foursquare ID
CLIENT_SECRET = '4AJRYELS00FWGQFH20DYLZ4VV2NGHK3JFZZUFS13A1IAKB0R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 500

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID = DE4QGIKFAN4PWTAPP1TH0I10F21BUIC3OYSSOUINDD2CKBL2, 
            CLIENT_SECRET = 3M1X5OEZ2RCPASJNH33SUXIU3EOOMXBZKS22JQG5FXEJ0IWT, 
            VERSION = 2019/04/22, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
%%capture
df_venues=getNearbyVenues(names=df_new['Neighbourhood'],latitudes=df_new['Latitude'],longitudes=df_new['Longitude']);

print(df_venues.shape)
df_venues.head()

In [16]:
df_venues_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")
df_venues_onehot.drop('Neighborhood',axis=1,inplace=True)

# add neighborhood column back to the front of dataframe
df_venues_onehot.insert(0,'Neighborhood',df_venues['Neighborhood'])

print(df_venues_onehot.shape)
df_venues_onehot.head()

(2258, 279)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern,Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Port Union,Rouge Hill,Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"West Hill,Morningside,Guildwood",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"West Hill,Morningside,Guildwood",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"West Hill,Morningside,Guildwood",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_grouped=df_venues_onehot.groupby('Neighborhood').mean().reset_index()
df_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,Caledonia-Fairbanks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0


In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
import numpy as np

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Sandwich Place,Breakfast Spot,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
1,Bayview Village,Chinese Restaurant,Japanese Restaurant,Café,Bank,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
2,Berczy Park,Coffee Shop,Cocktail Bar,Pub,Café,Restaurant,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Cheese Shop
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Garden,Garden Center,Butcher,Burrito Place,Auto Workshop,Fast Food Restaurant,Farmers Market,Spa,Restaurant
4,Caledonia-Fairbanks,Park,Pharmacy,Women's Store,Market,Fast Food Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


In [21]:
from sklearn.cluster import KMeans

In [22]:
kclusters=5
df_clustering = df_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(init='k-means++',n_clusters=kclusters,n_init=12).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 3, 0, 0, 0, 0, 0])

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,0,Lounge,Skating Rink,Sandwich Place,Breakfast Spot,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
1,Bayview Village,0,Chinese Restaurant,Japanese Restaurant,Café,Bank,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
2,Berczy Park,0,Coffee Shop,Cocktail Bar,Pub,Café,Restaurant,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Cheese Shop
3,Business Reply Mail Processing Centre 969 Eastern,0,Light Rail Station,Garden,Garden Center,Butcher,Burrito Place,Auto Workshop,Fast Food Restaurant,Farmers Market,Spa,Restaurant
4,Caledonia-Fairbanks,3,Park,Pharmacy,Women's Store,Market,Fast Food Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


In [24]:
df_geo2=df_new.iloc[:,2:5].rename(columns={'Neighbourhood':'Neighborhood'})
df_result=df_geo2.merge(neighborhoods_venues_sorted,on='Neighborhood')
df_result

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern,Rouge",43.806686,-79.194353,2,Fast Food Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Farmers Market
1,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497,1,Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Department Store
2,"West Hill,Morningside,Guildwood",43.763573,-79.188711,0,Rental Car Location,Breakfast Spot,Intersection,Mexican Restaurant,Medical Center,Electronics Store,Spa,Pizza Place,Donut Shop,Department Store
3,Woburn,43.770992,-79.216917,0,Coffee Shop,Pharmacy,Korean Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
4,Cedarbrae,43.773136,-79.239476,0,Athletics & Sports,Caribbean Restaurant,Bakery,Bank,Thai Restaurant,Fried Chicken Joint,Hakka Restaurant,Lounge,Eastern European Restaurant,Dumpling Restaurant
5,Scarborough Village,43.744734,-79.239476,0,Women's Store,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
6,"Kennedy Park,Ionview,East Birchmount Park",43.727929,-79.262029,0,Discount Store,Bus Station,Coffee Shop,Hobby Shop,Department Store,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Dive Bar
7,"Oakridge,Golden Mile,Clairlea",43.711112,-79.284577,0,Bakery,Soccer Field,Intersection,Bus Station,Bus Line,Metro Station,Fast Food Restaurant,Park,German Restaurant,General Travel
8,"Scarborough Village West,Cliffside,Cliffcrest",43.716316,-79.239476,0,Motel,American Restaurant,Yoga Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
9,"Cliffside West,Birch Cliff",43.692657,-79.264848,0,General Entertainment,College Stadium,Skating Rink,Café,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar


In [25]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [26]:


# create map
map_clusters = folium.Map(location=[43.741667, -79.373333], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_result['Latitude'], df_result['Longitude'], df_result['Neighborhood'], df_result['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



Cluster 0

In [27]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels']==0]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,0,Lounge,Skating Rink,Sandwich Place,Breakfast Spot,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
1,Bayview Village,0,Chinese Restaurant,Japanese Restaurant,Café,Bank,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
2,Berczy Park,0,Coffee Shop,Cocktail Bar,Pub,Café,Restaurant,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Cheese Shop
3,Business Reply Mail Processing Centre 969 Eastern,0,Light Rail Station,Garden,Garden Center,Butcher,Burrito Place,Auto Workshop,Fast Food Restaurant,Farmers Market,Spa,Restaurant
5,Canada Post Gateway Processing Centre,0,Coffee Shop,Hotel,American Restaurant,Sandwich Place,Gym / Fitness Center,Mediterranean Restaurant,Fried Chicken Joint,Burrito Place,Middle Eastern Restaurant,Dim Sum Restaurant
6,Cedarbrae,0,Athletics & Sports,Caribbean Restaurant,Bakery,Bank,Thai Restaurant,Fried Chicken Joint,Hakka Restaurant,Lounge,Eastern European Restaurant,Dumpling Restaurant
7,Central Bay Street,0,Coffee Shop,Italian Restaurant,Chinese Restaurant,Café,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Bar,Sandwich Place,Thai Restaurant
8,Christie,0,Café,Grocery Store,Park,Restaurant,Baby Store,Coffee Shop,Athletics & Sports,Nightclub,Convenience Store,Italian Restaurant
9,Church and Wellesley,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Pub,Café,Men's Store,Bubble Tea Shop
10,"Cliffside West,Birch Cliff",0,General Entertainment,College Stadium,Skating Rink,Café,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar


Cluster 1

In [28]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels']==1]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,"Port Union,Rouge Hill,Highland Creek",1,Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Department Store


Cluster 2

In [29]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels']==2]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,"Malvern,Rouge",2,Fast Food Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Farmers Market
59,"Silverthorn,Mount Dennis,Keelesdale,Del Ray",2,Fast Food Restaurant,Sandwich Place,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant


Cluster 3

In [30]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels']==3]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Caledonia-Fairbanks,3,Park,Pharmacy,Women's Store,Market,Fast Food Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
16,"Downsview East,CFB Toronto",3,Bus Stop,Park,Airport,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
20,East Toronto,3,Park,Convenience Store,Coffee Shop,Rental Car Location,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
34,Lawrence Park,3,Bus Line,Park,Swim School,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
43,"Old Mill North,Montgomery Road,The Kingsway",3,River,Park,Pool,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
47,Parkwoods,3,Fast Food Restaurant,Park,Food & Drink Shop,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
54,Rosedale,3,Park,Playground,Trail,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
64,"St. Phillips,Richview Gardens,Martin Grove Gar...",3,Pizza Place,Park,Bus Line,Mobile Phone Shop,Yoga Studio,Dive Bar,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
65,"Steeles East,Milliken,L'Amoreaux East,Agincour...",3,Coffee Shop,Playground,Park,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
82,"Upwood Park,North Park,Downsview",3,Park,Bakery,Construction & Landscaping,Basketball Court,Yoga Studio,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run


Cluster 4

In [31]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels']==4]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,"Humberlea,Emery",4,Baseball Field,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Department Store
70,"Sunnylea,Royal York South East,The Queensway E...",4,Baseball Field,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Department Store
